In [52]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import calendar
import sys
import json

class File_Transform():
    def __init__(self, path, input_dict):
        self.path = path
        self.error = ''
        self.general_dict = {
            'Дата платежа' : 'PAY_DATE',
            'Объем платежа' : 'PAY_AMOUNT',
            'ID пользователя' : 'USER_ID',
            'Дата регистрации' : 'REG_DATE',
            'Страна' : 'COUNTRY',
            'Источник' : 'SOURCE',
            'Платформа' : "PLATFORM"
        }
        
        
        self.general_dict_inv = {x : y for y,x in self.general_dict.items()}
        
        
        self.general_dtypes = {
             'PAY_DATE' : 'date',
            'PAY_AMOUNT' : 'float',
            'USER_ID' : 'str',
             'REG_DATE' : 'date',
            'COUNTRY' : 'str',
            'SOURCE' : 'str',
             "PLATFORM" : 'str'
        }
        self.input_dict = input_dict
    
    def check_separation(self):
        for sep in ['\t', ',', ';']:
            df = pd.read_csv(self.path, sep=sep)
            if df.shape[1]>1:
                self.df =  df
        print(self.df.head())
                
    def change_column_names(self):
        map_dict = {y : self.general_dict[x] for x,y in self.input_dict.items()}
        print(map_dict)
        for key in self.general_dtypes:
            if not key in map_dict.values():
                self.general_dtypes[key] = False
        self.df = self.df.rename(columns=map_dict)[list(map_dict.values())]
        
    
    def change_str_columns(self):
        for key in self.general_dtypes:
            if self.general_dtypes[key]=='str':
                self.df[key] = self.df[key].astype(str)

                
                
    def change_float_columns(self):
        for key in self.general_dtypes:
            if self.general_dtypes[key]=='float':
                try:
                    self.df[key] = self.df[key].astype(str).apply(lambda x: x.replace(',', '.')).astype(float)
                except:
                    self.path+=f"Problem with column : {self.general_dict_inv[key]}"
                    
                    
    def change_datetime_columns(self):
        timestamp = False
        for key in self.general_dtypes:
            if self.general_dtypes[key]=='date':
                try:
                    print(key)
                    self.df[key + '_TS'] = self.df[key].astype('float64').astype('float64')
                    timestamp = True
                except:
                    pass
                if not timestamp:
                    self.df[key] = self.df[key].astype(str).apply(lambda x: x.replace('.', '-')[0:10])
                    self.df[key + '_YEAR'] = self.df[key].astype(str).apply(lambda x: x[-4:])
                    try:
                        self.df[key + '_YEAR'] = self.df[key + '_YEAR'].astype(int)
                        self.df[key] = pd.to_datetime(self.df[key], format='%d-%m-%Y')
                        self.df = self.df.drop(key + '_YEAR', axis=1)
                    except:
                        self.df[key] = pd.to_datetime(self.df[key], format='%Y-%m-%d')
                    
                    self.df[key + '_TS'] = pd.to_timedelta(self.df[key], unit='ns').dt.total_seconds().astype(int)
                    
    def save_df(self):
        new_path = self.path.split('csv')[0]
        new_path = new_path[:-1] + '_transformed.csv'
        self.df.to_csv(new_path, index=False)
        
        
# if __name__ == "__main__":
    
    # path = sys.argv[0]
    # json_input_dict = sys.argv[1]
    # input_dict = (json.loads(json_input_dict))
    

#     sys.stdout.write(error)
#     sys.exit(0)

In [60]:
# import math

# math.log10(1598526054)

In [53]:
    path = "devtodev_Sql_wizard_27082020_0600.csv"
#     input_dict = {
#         'Дата платежа' : 'Date',
#         'Объем платежа' : 'SumRevenue_clean',
#         'ID пользователя' : 'BinDeviceID',
#         'Дата регистрации' : 'Cohort',
#         'Страна' : 'Country',
#         'Источник' : '_ms_common',
# #         'Платформа' : "PlatformType"
#     } 

    input_dict = {
        'Дата платежа' : 'payments.Date',
        'Объем платежа' : 'payments.Amount_in_USD',
        'ID пользователя' : 'users.devtodev_ID',
        'Дата регистрации' : 'users.Install_date',
#         'Страна' : 'Country',
#         'Источник' : '_ms_common',
#         'Платформа' : "PlatformType"
    } 

    
    
    Transf = File_Transform(path, input_dict)
    Transf.check_separation()
    Transf.change_column_names()
    Transf.change_str_columns()
    Transf.change_float_columns()
    Transf.change_datetime_columns()
    Transf.save_df()
    error = Transf.error

   payments.Amount_in_USD   payments.Transaction_ID  users.devtodev_ID  \
0               34.990000  GPA.3342-6219-7417-35915             115243   
1                1.990000  GPA.3369-9253-1954-34560             115417   
2                3.990000  GPA.3351-6437-1737-29874             115430   
3                3.990000  GPA.3314-0200-8080-35571             115430   
4                5.076454  GPA.3340-4230-5353-89827              70582   

   payments.Date  users.Install_date  
0   1.598410e+12        1.598410e+12  
1   1.598450e+12        1.598450e+12  
2   1.598460e+12        1.598450e+12  
3   1.598460e+12        1.598450e+12  
4   1.597070e+12        1.597070e+12  
{'payments.Date': 'PAY_DATE', 'payments.Amount_in_USD': 'PAY_AMOUNT', 'users.devtodev_ID': 'USER_ID', 'users.Install_date': 'REG_DATE'}
PAY_DATE
REG_DATE


In [54]:
# !python test.py devtodev_Sql.csv

In [77]:
# df = pd.read_csv("devtodev_Sql_wizard_27082020_0600_transformed.csv")
# print(df.dtypes, '=============\n\n',df.nunique())